In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
class JMTSolver(object):
    def __init__(self, a, b, v1, v2, t):
        self.c0 = a
        self.c1 = v1
        self.c2 = 0
        self.c3 = -2 * (5*a - 5*b + 3*t*v1 + 2*t*v2) / t**3
        self.c4 = (15*a - 15*b + 8*t*v1 + 7*t*v2) / t**4
        self.c5 = -3 * (2*a - 2*b + t*v1 + t*v2) / t**5
        
        #print "{}*s^5 + {}*s^4 + {}*s^3 + {}*s^2 + {}*s + {}".format(c5, c4, c3, c2, c1, c0)
        
    def f(self, x):
        return (self.c5 * x ** 5 + self.c4 * x ** 4 + self.c3 * x ** 3 
               + self.c2 * x ** 2 + self.c1 * x + self.c0)
    
    def fd(self, x):
        return (5 * self.c5 * x ** 4 + 4 * self.c4 * x ** 3 + 3 * self.c3 * x ** 2
               + 2 * self.c2 * x + self.c1)
    
    def fdd(self, x):
        return 20 * self.c5 * x ** 3 + 12 * self.c4 * x ** 2 + 6 * self.c3 * x + 2 * self.c2


class LinearSolver(object):
    def __init__(self, a, b, t):
        assert(t > 0)
        self.c0 = a
        self.c1 = (b - a) / t
        
    def f(self, x):
        return self.c1 * x + self.c0
    
    def fd(self, x):
        return self.c1
    
    def fdd(self, x):
        return 0
        
        
def curvature(solverx, solvery, t):
    return ((solverx.fd(t) * solvery.fdd(t) - solverx.fdd(t) * solvery.fd(t)) /
            (solverx.fd(t) ** 2 + solvery.fd(t) ** 2) ** 1.5)
        

T = 0.1
solverx = LinearSolver(0.0, 40, T)
solvery = JMTSolver(0.0, 4.0, 0.0, 0.0, T)
        
x = []
y = []
c = []

L = 100
for i in xrange(L+1):
    t = float(i) / L * T
    x.append(solverx.f(t))
    y.append(solvery.f(t))
    c.append(curvature(solverx, solvery, t))

data = []
data.append(go.Scatter(
    x=x,
    y=y,
    mode="lines+markers",
    text=map(str, c)
))

layout = go.Layout(
    height=900,
    yaxis={
        "scaleanchor": "x",
    },
)

figure = go.Figure(data=data, layout=layout)
iplot(figure, filename="JMT")